In [2]:
import json
from pathlib import Path
import random

BASE_DIR = Path.cwd().parent   # move up one level if notebook is inside src/
CONFIG_PATH = BASE_DIR / "config" / "skills_config.json"

with CONFIG_PATH.open() as f:
    CONFIG = json.load(f)

SKILLS = CONFIG["skills"]
POSITIONS = CONFIG["positions"]
DEFAULT_SKILL_VALUE = CONFIG["default_skill_value"]
PENALTIES = CONFIG["penalties"]

ALL_SKILLS = {
    skill_name: skill_data
    for group in CONFIG["skills"].values()
    for skill_name, skill_data in group.items()
}

skill_list = []
default_value = []

# get default values for skill ratings
for skill in SKILLS.keys():
    skill_list.append(skill)
    default_value.append(SKILLS.get(skill).get('base_factor'))

skills_dict = dict(zip(skill_list, default_value))

# Player Class
- allows me to create a player, with position, name, hometown, year (in school)

In [3]:
class Player:
    def __init__(self, name, hometown, hometown_state, position, height, weight, team, year):
        self.name = name
        self.hometown = hometown
        self.hometown_state = hometown_state
        self.position = position
        self.height = height
        self.weight = weight
        self.team = team
        self.year = year
        self.skills = {}

    def assign_skills(self):
        # Copy base skill factors
        factors = skills_dict.copy()

        # Apply position overrides (if any)
        position_overrides = CONFIG.get('positions').get(self.position,{}).get('overrides')
        factors.update(position_overrides)

        # Generate skills
        for skill, factor in factors.items():
            self.skills[skill] = int(
                DEFAULT_SKILL_VALUE * factor * random.uniform(0.9, 1.1)
            )

        for skill, value in self.skills.items():
            setattr(self, skill, value)

    def speak(self):
        print(
            f"My name is {self.name}. "
            f"I play {self.position} and I am a {self.year} "
            f"at the University of {self.team}."
            f"and my skills are {self.skills}."
        )

# put players into a dictionary
- makes things easier to sort

In [5]:
import pandas as pd

# List to store all created players
all_players = []

# Method to convert a Player instance to a dictionary
def player_to_dict(player):
    # Combine class attributes and skills into a single dictionary
    player_data = {
        "name": player.name,
        "hometown": player.hometown,
        "hometown_state": player.hometown_state,
        "position": player.position,
        "team": player.team,
        "year": player.year,
        **player.skills  # Spread skills into the dictionary
    }
    return player_data

# test that things are working

In [6]:
# def __init__(self, name, position,  height, weight, team, year):
# Example usage
player1 = Player("Tom Brady", "Grand Terrace", "CA", "QB", "75", "215", "Patriots", "Freshman")
player1.assign_skills()
all_players.append(player_to_dict(player1))  # Add player data as a dictionary
player1.speak()

player2 = Player("Joe Smith", "Colton", "CA", "OL", "78", "330", "Bears", "Freshman")
player2.assign_skills()
all_players.append(player_to_dict(player2))  # Add player data as a dictionary

print(f"{player1.name}'s skills: {player1.skills}")
print(f"{player2.name}'s skills: {player2.skills}")

My name is Tom Brady. I play QB and I am a Freshman at the University of Patriots.and my skills are {'speed': 54, 'run_blocking': 48, 'pass_blocking': 47, 'passing': 74, 'running': 50, 'catching': 48, 'tackling': 46, 'block_shedding': 53, 'pursuit': 46, 'zone_coverage': 47, 'man_coverage': 54}.
Tom Brady's skills: {'speed': 54, 'run_blocking': 48, 'pass_blocking': 47, 'passing': 74, 'running': 50, 'catching': 48, 'tackling': 46, 'block_shedding': 53, 'pursuit': 46, 'zone_coverage': 47, 'man_coverage': 54}
Joe Smith's skills: {'speed': 52, 'run_blocking': 56, 'pass_blocking': 68, 'passing': 47, 'running': 47, 'catching': 47, 'tackling': 47, 'block_shedding': 46, 'pursuit': 45, 'zone_coverage': 54, 'man_coverage': 46}


In [7]:
# Create a DataFrame from the list of player dictionaries
player_df = pd.DataFrame(all_players)
display(player_df)

,name,hometown,hometown_state,position,team,year,speed,run_blocking,pass_blocking,passing,running,catching,tackling,block_shedding,pursuit,zone_coverage,man_coverage
0,Tom Brady,Grand Terrace,CA,QB,Patriots,Freshman,54,48,47,74,50,48,46,53,46,47,54
1,Joe Smith,Colton,CA,OL,Bears,Freshman,52,56,68,47,47,47,47,46,45,54,46
